## Data Understanding
For this analysis, we will utilize a database from BLAH BLAH BLAH. It contains BLAH BLAH BLAH, Describe dataset.


In [1]:
#import necessary packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from random import gauss
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats as stats
from sklearn.feature_selection import RFE


%matplotlib inline

In [2]:
#ignore pairplot and graph warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
housing = pd.read_csv('data/kc_house_data.csv')
housing

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,8 Good,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,8 Good,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,7 Average,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,NONE,...,8 Good,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


In [4]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [5]:
housing.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [6]:
housing.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

Observations: We have several columns with categorical values
- waterfront      
- view             
- condition       
- grade          
- floors 
- bedrooms
- bathrooms
- zipcode

Also the column sqft_basement is an object. Let's find out why.

Also we can we can drop id and date sold column - since we arent comparing date listed to date sold, we can drop date sold column. Can also drop id column - as each listing represents a new sale.

We have several null values in the waterfont and yr_renovated categories. we have a small number of null values in the view category. Yr_renovated seems self-explanatory - a nan value means that the
house was not renovated. Let's explore the remaining two categories. 


In [ ]:
housing['waterfront'].unique()

In [8]:
housing['sqft_basement'].value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
2720.0        1
2390.0        1
243.0         1
1990.0        1
792.0         1
Name: sqft_basement, Length: 304, dtype: int64

In [ ]:
housing['view'].unique()

In [ ]:
housing['yr_renovated'].unique()

In [ ]:
housing['date'] =  pd.to_datetime(housing['date'])
housing

In [ ]:
max = housing['date'].max()
min = housing['date'].min()
print(min, max)

**Waterfront view**

- the only options are yes or no. 

We can either assume no waterfront view and replace all the nan values with no, or we can just drop the rows with a 
nan response. Or we can drop the entire column.
Since our data comprises of ~21,600 different rows, if we dropped these 2376 rows, we'd be loosing 
more than 10% of our data. 

So let's keep them and assume a nan response means no waterfront view. 

**View**

- several options - none, nan, good, excellent, average, and fair.

We can either assume a nan response means no view and replace all the nan values with none, 
or we can just drop the rows with a nan response. We could also drop the entire column. 
Since our data comprises of ~21,600 different rows, if we dropped these 63 rows, 
we'd only be loosing 0.3% of our data.

Let's keep them and assume a nan response means no view.

**Sqft_basement**
- let's drop the one row where the sqft_basement is a ?. 

## Data Preparation
- Encoding categorical values
- replacing null values in columns
- investigating duplicate id
- dropping columns

In [ ]:
housing['id'].duplicated().sum()

In [ ]:
housing['id'].duplicated()

In [ ]:
housing.loc[housing['id']== #####]

In [ ]:
housing.loc[housing['id']== #####]

In [ ]:
housing.drop(['id','date', 'lat', 'long'], axis=1, inplace=True)

In [ ]:
na_waterfront = housing[housing['waterfront'].isna()]
na_waterfront


In [ ]:
housing['waterfront'].fillna(value='NO', inplace=True)

In [ ]:
housing['view'].fillna(value='NONE', inplace=True)

In [ ]:
housing.head(10)

In [11]:
housing.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [9]:
#range 1-11. figure out 33 either use average after train/test split. or assume 33 is typo for 3
bedrooms = housing['bedrooms'].unique()
bathrooms = housing['bathrooms'].unique()
floors = housing['floors'].unique()
print (f"This is the list of bedrooms {bedrooms}")
print (f"This is the list of bathrooms {bathrooms}")
print (f"This is the list of floors {floors}")

This is the list of bedrooms [ 3  2  4  5  1  6  7  8  9 11 10 33]
This is the list of bathrooms [1.   2.25 3.   2.   4.5  1.5  2.5  1.75 2.75 3.25 4.   3.5  0.75 4.75
 5.   4.25 3.75 1.25 5.25 6.   0.5  5.5  6.75 5.75 8.   7.5  7.75 6.25
 6.5 ]
This is the list of floors [1.  2.  1.5 3.  2.5 3.5]


In [32]:
housing['lat'].min()

47.1559

In [24]:
housing['zipcode'].value_counts()

98103    602
98038    589
98115    583
98052    574
98117    553
        ... 
98102    104
98010    100
98024     80
98148     57
98039     50
Name: zipcode, Length: 70, dtype: int64

In [ ]:
#Setting up train test split
X = housing.drop('price', axis=1)
y = housing['price']

X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=57)

In [ ]:
Housing_train= 

In [ ]:
lb_make = Labelencoder()


EXPLORATORY

In [ ]:
sns.pairplot(housing)
plt.show()

In [ ]:
housing.corr()

In [ ]:
housing.corr().price.sort_values(ascending=False)

In [ ]:
Simple Model - no encoding

In [ ]:
housing['sqft_above'].sort_values()

In [ ]:
#Multiple Regression model all variables (no encoding)
housing1 = housing.drop(['id','date', 'grade', 'condition', 'waterfront', 'view', 'sqft_basement', 'bedrooms', 'bathrooms', 'floors', 'yr_renovated', 'zipcode', 'lat', 'long'], axis=1)
housing1.head()

In [ ]:
lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select =4)

In [ ]:
ss = StandardScaler()
ss.fit(housing1.drop('price', axis=1))
housing1_scaled = ss.transform(housing1.drop('price', axis=1))

In [ ]:
housing1_scaled

In [ ]:
select.fit(X=housing1_scaled, y=housing1['price'])

In [ ]:
select.support_

In [ ]:
select.ranking_

In [ ]:
## use sqft_living  yr_built  sqft_living15  sqft_lot15

In [ ]:
Polynomials using all except categorical values

In [ ]:
X = housing1.drop('price', axis=1)
y = housing1['price']
pf = PolynomialFeatures(degree=3)
pf.fit(X)

In [ ]:
pf.transform(X)

In [ ]:
pf.transform(X).shape

In [ ]:
pf.get_feature_names()

In [ ]:
polynomial_df = pd.DataFrame(pf.transform(X), columns= pf.get_feature_names() )

In [ ]:
lr = LinearRegression()
lr.fit(polynomial_df, y)

In [ ]:
lr.score(polynomial_df, y)

## DO NOT RUN CELL BELOW 

In [ ]:
sns.pairplot(housing, 'price')

In [ ]:
housing.corr().price.sort_values(ascending=False)

In [ ]:
mask = np.triu(np.ones_like(housing.corr(), dtype=bool))

plt.figure(figsize=(20,10))

cor = housing.corr().abs()
sns.heatmap(cor, mask=mask, annot=True);